<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/PiScorer_as_GRPO_Reward_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://play.withpi.ai/logo/logoFullBlack.svg" width="240"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://build.withpi.ai"><font size="4">Copilot</font></a>

Are you constantly relying on LLM-as-a-judge to evaluate your model’s performance?

Have you ever wanted to assess your model at every training checkpoint but hesitated because LLM-as-a-judge is too slow and expensive?

**Now you can — with [Pi-Scorer](https://build.withpi.ai).**

[Pi-Scorer](https://build.withpi.ai) offers an alternative to LLM-as-a-judge with several advantages:

* Significantly faster

* Highly consistent — always returns the same score for the same inputs

* Eliminates the need for prompt tuning or adjustments

In this Colab, we integrate [Pi-Scorer](https://build.withpi.ai) as the reward function within the [Unsloth](https://unsloth.ai/) GRPO training loop, based on the [Unsloth Qwen2.5_(3B)-GRPO.ipynb colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen2.5_(3B)-GRPO.ipynb) notebook.

### Installation

In [1]:
from google.colab import userdata
import os

# Get PI API key: https://build.withpi.ai/account/keys
os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Load Unsloth Model

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-30 16:20:38 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-30 16:20:38 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.43%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness =

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Preparation and PI Reward Functions

In [5]:
from datasets import load_dataset, Dataset
import requests

# Load and prep dataset
SYSTEM_PROMPT = """
Generate a short TLDR of a subreddit post without any surrounding text. Here are some requirement of the TLDR:
1. Make sure that the TLDR is short and concise.
2. Make sure that the TLDR state the important points of the post
3. Make sure that the TLDR should make sense on its own.
"""

dataset = load_dataset("trl-lib/tldr", split="train")
dataset = dataset.remove_columns(["completion"])
dataset = dataset.rename_column("prompt", "post")
dataset = dataset.select(range(500))
dataset = dataset.map(
    lambda x: {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": x["post"]},
        ]
    }
)
print(dataset[0])


# Pi constants
PI_API_URL = "https://api.withpi.ai/v1/scoring_system/score"
HEADERS = {
    "Content-Type": "application/json",
    "x-api-key": os.environ.get("WITHPI_API_KEY"),
}

# Pi util functions
def get_pi_score(input: str, output: str, question: str) -> float:
    payload = {
        "llm_input": input,
        "llm_output": output,
        "scoring_spec": [{"question": question}]
    }
    # Can add retry if needed.
    response = requests.post(PI_API_URL, headers=HEADERS, json=payload)
    return response.json()["total_score"]

def score_tldrs(prompts, completions, question: str) -> list[float]:
    posts = [prompt[-1]["content"] for prompt in prompts]
    tldrs = [completion[0]["content"] for completion in completions]
    return [get_pi_score(post, tldr, question) for post, tldr in zip(posts, tldrs)]

# Reward functions
def pi_concise(prompts, completions, **kwargs) -> list[float]:
    return score_tldrs(prompts, completions, "Is the TLDR concise and to the point?")

def pi_coverage(prompts, completions, **kwargs) -> list[float]:
    return score_tldrs(prompts, completions, "Does the TLDR state the important points of the post?")

def pi_standalone(prompts, completions, **kwargs) -> list[float]:
    return score_tldrs(prompts, completions, "Does the TLDR make sense on its own without needing to refer to the original post?")

README.md:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116722 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6447 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6553 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'post': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mot

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [6]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 40,
    save_steps = 10,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [7]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        pi_concise,
        pi_coverage,
        pi_standalone,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 119,734,272/3,000,000,000 (3.99% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / pi_concise,rewards / pi_coverage,rewards / pi_standalone
1,-0.000000,2.771851,0.495291,45.500000,0.000000,0.972656,0.926270,0.872925
2,-0.000000,2.937988,0.118840,61.500000,0.000000,0.999023,0.957520,0.981445
3,0.000000,2.813477,0.117387,48.000000,0.001235,0.993652,0.848145,0.971680
4,0.000100,2.659180,0.238640,70.250000,0.001317,0.957520,0.857910,0.843750
5,0.000100,2.917969,0.143859,54.875000,0.001675,0.992676,0.973145,0.952148
6,0.000100,2.981934,0.038835,64.750000,0.001454,0.999512,0.985840,0.996582
7,0.000000,2.819336,0.193808,47.750000,0.001222,0.983398,0.949219,0.886719
8,0.000000,2.989746,0.015616,48.500000,0.000882,1.000000,0.991699,0.998047
9,0.000100,2.939941,0.076116,60.125000,0.001466,0.991211,0.990723,0.958008
10,0.000100,2.816406,0.209225,73.625000,0.001722,0.951660,0.943359,0.921387


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=40, training_loss=7.163009181216928e-05, metrics={'train_runtime': 466.9086, 'train_samples_per_second': 0.685, 'train_steps_per_second': 0.086, 'total_flos': 0.0, 'train_loss': 7.163009181216928e-05})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [8]:
post = """SUBREDDIT: r/relationships

TITLE: Me [19 F] with my friend [19 M], not sure if I may have messed things up already.

POST: Hello hello everybody. I hope this isn't too trivial of a question to ask on here, but I've been feeling a bit out of my depth when it comes to this situation (I've had only one relationship before, and for many reasons, it was out of the ordinary).

Okay! So, a couple of weeks ago, I started talking to this guy on Facebook, through a student group that we were both part of. I thought he was sort of cute, so I sent him a PM just to talk, etc, etc. We're both transfer students at the same school, so I knew that we could eventually meet in person once we both moved on-campus. So, we did, and we hung out maybe twice, just as friends.

Okay. So, everything is going pretty well. We talk over Facebook and Snapchat, whatever. So, Saturday night, I was just hanging out with people and kind of being bored, when I got a Snapchat from him asking what I was doing. I asked if he wanted to hang out, so we did.

We ended up smoking pot (the first time for me, ever), and sort of just wandering around. Eventually we ended up back at his dorm room, where high me decided to just go for it, and I came on to him pretty strongly. It worked out for me (luckily, otherwise things would have been really super awkward), and we ended up messing around but not having sex.

Yesterday, however, I ended up going to hang out with him again, and this time we did sleep together. Afterward, we kind of discussed what we were going to do, and he just said that he wanted to "play it by ear" and not slap any labels on anything. I'm wondering if this means that he wants a fwb-type situation, or if he might actually be interested in me. The way I've been acting is extremely out of character for me, and I am not interested in having a fuck buddy. I like him, and I would be very interested in maybe seeing where things go, but I'm worried that I may have ruined my chances of a relationship by sleeping with him already.

TL;DR:
"""

text = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": post},
    ],
    tokenize=False,
    add_generation_prompt=True,
)

from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.4,
    top_p=0.95,
    max_tokens=1024,
)
output = (
    model.fast_generate(
        [text],
        sampling_params=sampling_params,
        lora_request=None,
    )[0]
    .outputs[0]
    .text
)

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'The OP has had a one-night stand with a friend and is unsure if they have ruined their chances of a relationship. They are worried about the implications of sleeping with a friend and want to know if he wants a casual relationship or something more serious.'

Now we load the LoRA and test:

In [13]:
model.save_lora("grpo_saved_lora")

output = (
    model.fast_generate(
        [text],
        sampling_params=sampling_params,
        lora_request=None,
    )[0]
    .outputs[0]
    .text
)

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'The OP is concerned that they may have ruined their chances for a relationship by sleeping with their friend after a casual hookup, and are unsure if their friend wants a casual relationship or something more serious.'